# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
#Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [3]:
#Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [4]:
#Your code here; use a subquery. No join will be necessary.
c.execute('''SELECT customerNumber, contactLastName, contactFirstName
             FROM customers
             WHERE customerNumber in (SELECT customerNumber FROM orders WHERE orderDate = '2003-01-31');''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

customerNumber contactLastName contactFirstName
0            141          Freyre           Diego

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [11]:
#Your code here
c.execute('''SELECT productName, sum(quantityOrdered) as total_ordered
             FROM products
             JOIN orderdetails
             USING(productCode)
             GROUP BY 1
             ORDER BY total_ordered DESC;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

productName  total_ordered
0              1992 Ferrari 360 Spider red           1808
1                     1937 Lincoln Berline           1111
2                American Airlines: MD-11S           1085
3  1941 Chevrolet Special Deluxe Cabriolet           1076
4             1930 Buick Marquette Phaeton           1074

In [11]:
#Your code here
c.execute('''SELECT productName, sum(quantityOrdered) as total_ordered
             FROM products
             JOIN offices
             USING(officeCode)
             GROUP BY 1
             ORDER BY total_ordered DESC;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

productName  total_ordered
0              1992 Ferrari 360 Spider red           1808
1                     1937 Lincoln Berline           1111
2                American Airlines: MD-11S           1085
3  1941 Chevrolet Special Deluxe Cabriolet           1076
4             1930 Buick Marquette Phaeton           1074

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [13]:
#Your code here
c.execute('''SELECT productName, count(DISTINCT customerNumber) as total_customers
             FROM products
             JOIN orderdetails
             USING(productCode)
             JOIN orders
             USING(orderNumber)
             GROUP BY 1
             ORDER BY total_customers DESC;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

productName  total_customers
0  1992 Ferrari 360 Spider red               40
1           1934 Ford V8 Coupe               27
2     1952 Alpine Renault 1300               27
3          1972 Alfa Romeo GTA               27
4             Boeing X-32A JSF               27

## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [25]:
#Your code here
c.execute('''SELECT employeeNumber, officeCode, city, firstName, lastName
             FROM employees
             JOIN offices
             USING(officeCode)
             WHERE employeeNumber in (SELECT salesRepEmployeeNumber
                                      FROM customers
                                      JOIN orders
                                      USING(customerNumber)
                                      GROUP BY 1
                                      HAVING count(DISTINCT customerNumber) <= 20)
             ORDER BY employeeNumber ASC;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

employeeNumber officeCode           city firstName   lastName
0            1165          1  San Francisco    Leslie   Jennings
1            1166          1  San Francisco    Leslie   Thompson
2            1188          2         Boston     Julie   Firrelli
3            1216          2         Boston     Steve  Patterson
4            1286          3            NYC  Foon Yue      Tseng
5            1323          3            NYC    George     Vanauf
6            1337          4          Paris      Loui     Bondur
7            1370          4          Paris    Gerard  Hernandez
8            1401          4          Paris    Pamela   Castillo
9            1501          7         London     Larry       Bott
10           1504          7         London     Barry      Jones
11           1611          6         Sydney      Andy     Fixter
12           1612          6         Sydney     Peter      Marsh
13           1621          5          Tokyo      Mami      Nishi
14           1702          4          Paris    Martin     Gerard

In [27]:
# OR

# c.execute("""select distinct employeeNumber,
#                     officeCode,
#                     o.city,
#                     firstName,
#                     lastName
#                     from employees e
#                     join offices o
#                     using(officeCode)
#                     join customers c
#                     on e.employeeNumber = c.salesRepEmployeeNumber
#                     join orders
#                     using (customerNumber)
#                     join orderdetails
#                     using (orderNumber)
#                     where productCode in (select productCode
#                                                  from products
#                                                  join orderdetails
#                                                  using (productCode)
#                                                  join orders
#                                                  using (orderNumber)
#                                                  group by 1
#                                                  having count(distinct customerNumber) <= 20
#                                          )
#                     ORDER BY employeeNumber ASC;
#            """
#          )
# df = pd.DataFrame(c.fetchall())
# df.columns = [x[0] for x in c.description]
# df

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [31]:
#Your code here
c.execute('''SELECT employeeNumber, firstName, lastName, count(DISTINCT customerNumber) as total_customers
             FROM employees as e
             JOIN customers as c
             ON e.employeeNumber = c.salesRepEmployeeNumber
             GROUP BY 1
             HAVING avg(creditLimit) > 15000;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

employeeNumber firstName   lastName  total_customers
0            1165    Leslie   Jennings                6
1            1166    Leslie   Thompson                6
2            1188     Julie   Firrelli                6
3            1216     Steve  Patterson                6
4            1286  Foon Yue      Tseng                7
5            1323    George     Vanauf                8
6            1337      Loui     Bondur                6
7            1370    Gerard  Hernandez                7
8            1401    Pamela   Castillo               10
9            1501     Larry       Bott                8
10           1504     Barry      Jones                9
11           1611      Andy     Fixter                5
12           1612     Peter      Marsh                5
13           1621      Mami      Nishi                5
14           1702    Martin     Gerard                6

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!